<a href="https://colab.research.google.com/github/wangbxj1234/5001/blob/main/voxelize%26intomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import torch

In [25]:
def collate_fn(coord, feat, label):#(B,N,C)
    #coord, feat, label = list(zip(*batch))#(B,N,C)
    offset = [6,12]
    
    ####
    #coord, feat, label = torch.cat(coord,dim=0), torch.cat(feat,dim=0), torch.cat(label,dim=0) #但是属于不同batch的，就算坐标近也不能分进一个v。
    offset = torch.IntTensor(offset)##(N,3),(N,C),(N,),O

    ##to np
    coord, feat, label = coord.cpu().detach().numpy(),feat.cpu().detach().numpy(),label.cpu().detach().numpy()
    ##
    PN=0;
    voxel=[]
    for o in offset:
        c, f, l =coord[PN:int(o),:],feat[PN:int(o),:],label[PN:int(o),:]#(n,C),(n,C),(n,)
        discrete_coord = np.floor(c *4) 
        assert discrete_coord.ndim == 2
    # Floor first for negative coordinates
        arr = discrete_coord.copy()
        arr = arr.astype(np.uint64, copy=False)
        hashed_arr = np.uint64(14695981039346656037) * np.ones(arr.shape[0], dtype=np.uint64)
        for j in range(arr.shape[1]):
            hashed_arr *= np.uint64(1099511628211)
            hashed_arr = np.bitwise_xor(hashed_arr, arr[:, j])
        idx_sort = np.argsort(hashed_arr) 
        key_sort = hashed_arr[idx_sort] 
        _, count1 = np.unique(key_sort, return_counts=True)
        coord[PN:int(o),:],feat[PN:int(o),:],label[PN:int(o),:]= c[idx_sort],f[idx_sort],l[idx_sort]
        voxel.append(count1)
        PN=int(o)

    ##to torch
    coord, feat, label = torch.tensor(coord),torch.tensor(feat),torch.tensor(label)
    ##    

    return coord, feat, label, offset, voxel

In [26]:
#coord, feat, label=torch.rand(2,6,3),torch.rand(2,6,4),torch.rand(2,6,1)
coord, feat, label=torch.rand(2*6,3),torch.rand(2*6,32),torch.rand(2*6,1)
coord[0,:]=coord[1,:]
coord

tensor([[0.2378, 0.6692, 0.8957],
        [0.2378, 0.6692, 0.8957],
        [0.7604, 0.7277, 0.9184],
        [0.8798, 0.6896, 0.3182],
        [0.2042, 0.1435, 0.4127],
        [0.2708, 0.1910, 0.9157],
        [0.5578, 0.7715, 0.8379],
        [0.7552, 0.1765, 0.9433],
        [0.9547, 0.1291, 0.8407],
        [0.9380, 0.7244, 0.2583],
        [0.6284, 0.5067, 0.1116],
        [0.9409, 0.9553, 0.2261]])

In [27]:
coord, feat, label, offset, voxel = collate_fn(coord, feat, label)
print(coord.size(),feat.size(),label.size())

torch.Size([12, 3]) torch.Size([12, 32]) torch.Size([12, 1])


In [28]:
voxel

[array([1, 1, 1, 1, 2]), array([2, 1, 1, 1, 1])]

In [29]:
voxel2=np.concatenate(voxel, axis=0)# 6-1=5
torch.tensor(voxel2)#好像实际操作只需要 voxel-wies的分界符，不需要 pcloud-wise的。

tensor([1, 1, 1, 1, 2, 2, 1, 1, 1, 1])

In [30]:
#好像只需要 voxel-wise的分界符，不需要 pcloud-wise的。
voxel3=np.cumsum(voxel2)
vo=torch.IntTensor(voxel3)#torch.IntTensor

then. into model

In [38]:
import torch
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from typing import Optional
from torch import nn


class linear_Attention(nn.Module):
    # flow attention in normal version
    def __init__(self, in_planes):
        super(linear_Attention, self).__init__()
        self.n_heads =8
        self.query_projection = nn.Linear(in_planes, in_planes)
        self.key_projection = nn.Linear(in_planes, in_planes)
        self.value_projection = nn.Linear(in_planes, in_planes)
        self.out_projection = nn.Linear(in_planes, in_planes)
        self.dropout = nn.Dropout(0.05)
        self.eps = 1e-6

    def kernel_method(self, x):
        return torch.sigmoid(x)

    def dot_product(self, q, k, v):
        kv = torch.einsum("hld,hlm->hdm", k, v)
        qkv = torch.einsum("hld,hdm->hlm", q, kv)
        return qkv

    def forward(self, pxo) -> torch.Tensor:
        p, x, o = pxo   
        PN=0;
        x_sub=[]
        p_sub=[]
        out=[]
        for offset in o:
           x_s=x[PN:int(offset),:]
           x_sub.append(x_s)
           tgt_len, embed_dim = x_s.size() ###(N,d) 
           #print(offset,tgt_len, embed_dim)
           PN=int(offset)
        ## input: (L or S) D; output: L D
        ## Note: queries, keys, values are not projected yet
        ## 1. Linear projection
           L=tgt_len
           print('L',L)
           S=tgt_len
           Q = self.query_projection(x_s).view(L, self.n_heads, -1)
           K = self.key_projection(x_s).view(S, self.n_heads, -1)
           V = self.value_projection(x_s).view(S, self.n_heads, -1)
           #Q = queries.transpose(0, 1)
           #K = keys.transpose(0, 1)
           #V = values.transpose(0, 1)
           # 2. Non-negative projection
           Q = self.kernel_method(Q)
           K = self.kernel_method(K)

           KV = torch.einsum("shd,shm->hmd", K, V)

           # Compute the normalizer
           Z = 1/(torch.einsum("lhd,hd->lh", Q, K.sum(dim=0))+self.eps)

           # Finally compute and return the new values
           V = torch.einsum("lhd,hmd,lh->lhm", Q, KV, Z)

           output = V.reshape(L, -1)
           output = self.out_projection(output)
           output = self.dropout(output)

           out.append(output)
        #out=torch.IntTensor(out)
        out=torch.cat(out,dim=0)

        return out



In [39]:
planes=32
transformer2 = linear_Attention(planes)
right_res = transformer2([coord, feat, vo])
print(right_res.shape)

L 1
L 1
L 1
L 1
L 2
L 2
L 1
L 1
L 1
L 1
torch.Size([12, 32])
